## Минимальный необходимый размер выборки

In [359]:
from statsmodels.stats import proportion
from scipy import stats

In [360]:
var_control = 2000 # стандартное отклонение контрольной группы
var_test = 2000    # стандартное отклонение тестовой группы
za_2 = stats.norm.ppf(0.025) # Квантинь стандартного норм. распределения, соответствующий половине вероятности ошибки первого рода
zb = stats.norm.ppf(0.2) # 0.2-квантиль стандартного нормального распределения
mde = 60     # Minimum Detectable Effect. Минимальный размер эффекта, который мы хотим зафиксировать в рамках AB теста. Здесь 60 рублей. 

#минимальный размер выборки
n = ((var_control + var_test) * (za_2 + zb)**2)/mde**2
n = round(n)
print(f'n: {n}')

n: 9


### Мин. размер выборки по историческим данным
Берём предыдущий период, считаем по нему дисперсию. В mde закладываем выгоду (эффект), которую хотим получить.


Объяснение формулы:
По нашим оценкам, дисперсия в контрольной и тестовой группах будет примерно равна 1 000 000 руб². 
Когда мы будем рассчитывать t-статистику, мы планируем использовать уровень значимости, равный 5%. Мы надеемся, что эффект от внедрения фичи будет равен +12.9₽ выручки на пользователя. Если это действительно так, то мы бы очень хотели, чтобы вероятность зафиксировать статистически значимое отклонение была равна 80%. 
Выдай, пожалуйста, нам такой размер выборки, чтобы удовлетворял всем нашим прихотям.

In [361]:
var_historical = 2500  # Дисперсия на исторических данных
za_2 = -1.96
zb = -1.282
mde = 1.5               # Минимальный размер эффекта, который мы хотим зафиксировать в рамках AB теста. Здесь 60 рублей
n = ((2*var_historical) * (za_2 + zb)**2)/mde**2
n = round(n)
print(f'n: {n}')

n: 23357


### Расчёт длительности эксперимента

In [362]:
a = 0.05    # Уровень значимости
P = 0.8     # Мощность

## Далее подставляем значения из статистики предыдущих периодов так, чтобы полученное значение относительного эффекта 
## было наиболее близким к желаемому, но не больше него. 
wanted_effect = 1.5 # Желаемый эффект в процентах
var_historical = 0.048  # дисперсия, оценённая на данных за прошлые периоды
n_historical = 2645900        # количество пользователей, которые в среднем посещают сайт за период, равный выбранной длительности
avg_val = 0.051 # среднее значение целевой метрики. Например, ARPU=6448 рублей

za_2 = stats.norm.ppf(a/2)
zb = stats.norm.ppf(1-P)
mde = -(za_2 + zb) * (4*var_historical/n_historical)**0.5
mde_relative = mde/avg_val*100

print(f'mde: {mde}')
print(f'mde_relative: {mde_relative} %')

if (mde_relative > wanted_effect):
    print('Такой период не подходит')
else:
    print('Период подходит. Ура!')

mde: 0.0007546885108949907
mde_relative: 1.4797813939117466 %
Период подходит. Ура!


### Дисперсия для конверсионных метрик

In [363]:
p = 0.04    # конверсия (вероятность успеха)
var_bernoulli = p*(1-p)
var_bernoulli

0.0384

### Мин. размер выборки для конверсионных метрик

In [364]:
p_historical = 0.04    # конверсия (вероятность успеха)
var_historical = p*(1-p)
za_2 = -1.96
zb = -0.842
mde = 60               # Минимальный размер эффекта, который мы хотим зафиксировать в рамках AB теста. Здесь 60 рублей
n = ((2*var_historical) * (za_2 + zb)**2)/mde**2
#n = round(n)
print(f'var_historical: {var_historical}')
print(f'n: {n}')

var_historical: 0.0384
n: 0.000167492352


# AB тест количественных метрик
Полезняшка: https://www.scaler.com/topics/data-science/a-b-testing-in-python/

In [365]:
x1 = 74.69  # среднее по тестовой выборке
x2 = 71.28 # среднее по контрольной выборке

sx1 = 21 # отклонение по тестовой выборке
sx2 = 25 # отклонение по контрольной выборке
m1_m2 = 0 # гипотеза
n = 2100 # количество измерений
a = 0.05

ese = ( (sx1**2/n) + (sx2**2/n) ) ** 0.5
ese = round(ese, 2)
print(f'ese: {ese}')

t = ((x1 - x2) - (m1_m2))/ ese
t = round(t, 2)

df = ((n-1)*(sx1**2 + sx2**2)**2)/(sx1**4 + sx2**4) # степени свободы
df = round(df)

p_value = round(stats.t.sf(abs(t), df=df)*2, 4)

print(f't: {t}')
print(f'df: {df}')
print(f'p_value: {p_value}')

if (p_value < a):
    print('Отвергаем гипотезу, что H1=H2. Результат статистически значимый. Если изменение желаемое, — раскатываем фичу')
else:
    print('Принимаем гипотезу, что H1=H2. Результат статичтически НЕ значимый')

ese: 0.71
t: 4.8
df: 4077
p_value: 0.0
Отвергаем гипотезу, что H1=H2. Результат статистически значимый. Если изменение желаемое, — раскатываем фичу


# Тест для пропорций

In [366]:
p1 = 0.1526   # доля успеха в тестовой группе
p2 = 0.1524  # доля успеха во второй группе
n1 = 7863369  # количество в первой группе
n2 = 7864235   # количество во второй группе
a = 0.05
successes = [n1*p1, n2*p2]
nobs = [n1, n2]
z_stat, p_value = proportion.proportions_ztest(successes, nobs)
p_value = round(p_value, 5)
print(f'p_value: {p_value}')
print(f'z_stat: {z_stat}')

if (p_value < a):
    print('Отвергаем гипотезу, что H1=H2. Результат статистически значимый. Если изменение желаемое, — раскатываем фичу')
else:
    print('Принимаем гипотезу, что H1=H2. Результат статичтически НЕ значимый')

p_value: 0.26997
z_stat: 1.1031295371812668
Принимаем гипотезу, что H1=H2. Результат статичтически НЕ значимый
